In [1]:
from planning_inference.parsers import parse_model, parse_problem, parse_plan, parse_trajectory, parse_hypothesis, parse_observation_sequence
from planning_inference.generator import generate_trajectory
from planning_inference.functions import generate_all_literals, get_matching_literals

from planning_inference.pddl import Conjunction, Literal, Type, TypedObject, Effect, Truth, NumericConstant, PrimitiveNumericExpression, Increase
from planning_inference.pddl import SensorModel

from planning_inference.observations import Trajectory, Hypothesis, State

from planning_inference import DecodingTask

from sensor_models import load_sensor_model

import os
import copy
from collections import defaultdict
from itertools import combinations
from random import choice, choices, shuffle
from statistics import mean
import glob
import time

In [2]:
def launch_experiments(domain, task, timeout):
    Ms = load_sensor_model(domain)
    
    observabilities = [30, 50, 70]
       
    for observability in observabilities:
        base_path = "benchmark/%s/%s/%s/" % (domain, task, str(observability))

        problems = sorted(glob.glob(base_path + "*"))

        for problem in problems:
            costs = []
            times = []

            print(problem)

            # Planning Model
            Mp = parse_model(problem + "/domain")
            
            # Real Hypothesis
            with open(problem + "/sol", "r") as f:
                correct_h = int(f.read())

            print("Correct hypothesis: %d" % correct_h)

            # Observation sequence
            obs = parse_observation_sequence(problem + "/obs")

            # Hypotheses
            h_files = sorted(glob.glob(problem + "/hyp*"))

            for i in range(len(h_files)):
                h = parse_hypothesis(h_files[i])

                #Build a decoding problem for each hypothesis
                T = DecodingTask(Mp,Ms,obs,h)

                tic = time.time()
                sol = T.decode(clean=True, planner="downward", t=timeout)
                toc = time.time()


                if len(sol.actions) == 0:
                    cost = 1000
                else:
                    cost = len(sol.actions)
                    sol.to_file(problem + "/plan.%s" % str(i).zfill(2))

                duration = toc - tic

                print("Hypothesis %d: %d, %.2f" % (i, cost, duration))

                costs.append(cost)
                times.append(duration)

            with open(problem + "/costs", "w") as f:
                f.write(" ".join(map(str, costs)))

            with open(problem + "/times", "w") as f:
                f.write(" ".join(map(str, times)))
    

In [3]:
domain = "openstacks"
timeout = 120

In [4]:
# MONITORING

task = "monitoring"
launch_experiments(domain, task, timeout)

benchmark/openstacks/monitoring/30/P00
Correct hypothesis: 4
Hypothesis 0: 20, 1.04
Hypothesis 1: 20, 1.02
Hypothesis 2: 1000, 0.87
Hypothesis 3: 1000, 0.88
Hypothesis 4: 20, 1.01
Hypothesis 5: 20, 1.03
benchmark/openstacks/monitoring/30/P01
Correct hypothesis: 2
Hypothesis 0: 1000, 1.32
Hypothesis 1: 1000, 1.41
Hypothesis 2: 21, 1.51
Hypothesis 3: 1000, 1.09
Hypothesis 4: 21, 1.51
Hypothesis 5: 1000, 1.21
benchmark/openstacks/monitoring/30/P02
Correct hypothesis: 3
Hypothesis 0: 18, 1.48
Hypothesis 1: 1000, 1.14
Hypothesis 2: 1000, 1.14
Hypothesis 3: 18, 1.39
Hypothesis 4: 18, 1.49
benchmark/openstacks/monitoring/30/P03
Correct hypothesis: 4
Hypothesis 0: 1000, 0.89
Hypothesis 1: 18, 0.95
Hypothesis 2: 20, 1.01
Hypothesis 3: 22, 1.09
Hypothesis 4: 18, 0.95
Hypothesis 5: 1000, 0.90
benchmark/openstacks/monitoring/30/P04
Correct hypothesis: 2
Hypothesis 0: 1000, 0.94
Hypothesis 1: 18, 1.03
Hypothesis 2: 18, 0.94
Hypothesis 3: 18, 0.95
Hypothesis 4: 1000, 0.89
Hypothesis 5: 1000, 0.88
be

In [5]:
# PREDICTION

task = "prediction"
launch_experiments(domain, task, timeout)

benchmark/openstacks/prediction/30/P00
Correct hypothesis: 3
Hypothesis 0: 45, 9.10
Hypothesis 1: 44, 2.02
Hypothesis 2: 46, 2.21
Hypothesis 3: 44, 2.23
Hypothesis 4: 46, 1.99
Hypothesis 5: 45, 12.83
benchmark/openstacks/prediction/30/P01
Correct hypothesis: 0
Hypothesis 0: 42, 2.23
Hypothesis 1: 43, 2.21
Hypothesis 2: 42, 2.24
Hypothesis 3: 43, 2.21
Hypothesis 4: 43, 2.21
Hypothesis 5: 42, 2.22
benchmark/openstacks/prediction/30/P02
Correct hypothesis: 1
Hypothesis 0: 45, 2.76
Hypothesis 1: 44, 2.70
Hypothesis 2: 44, 2.66
Hypothesis 3: 44, 2.68
Hypothesis 4: 45, 2.68
Hypothesis 5: 44, 2.69
benchmark/openstacks/prediction/30/P03
Correct hypothesis: 3
Hypothesis 0: 46, 2.06
Hypothesis 1: 45, 1.98
Hypothesis 2: 46, 2.00
Hypothesis 3: 44, 2.14
Hypothesis 4: 45, 2.04
Hypothesis 5: 46, 1.98
benchmark/openstacks/prediction/30/P04
Correct hypothesis: 3
Hypothesis 0: 45, 2.01
Hypothesis 1: 46, 1.96
Hypothesis 2: 46, 2.00
Hypothesis 3: 44, 2.01
Hypothesis 4: 44, 2.05
Hypothesis 5: 44, 2.02
benc

In [6]:
# HINDSIGHT

task = "hindsight"
launch_experiments(domain, task, timeout)

benchmark/openstacks/hindsight/30/P00
Correct hypothesis: 4
Hypothesis 0: 1000, 2.01
Hypothesis 1: 1000, 1.92
Hypothesis 2: 1000, 2.14
Hypothesis 3: 1000, 1.23
Hypothesis 4: 46, 2.32
Hypothesis 5: 1000, 1.49
benchmark/openstacks/hindsight/30/P01
Correct hypothesis: 0
Hypothesis 0: 36, 1.30
Hypothesis 1: 1000, 1.49
Hypothesis 2: 36, 1.34
Hypothesis 3: 1000, 1.85
Hypothesis 4: 1000, 1.51
benchmark/openstacks/hindsight/30/P02
Correct hypothesis: 0
Hypothesis 0: 45, 2.51
Hypothesis 1: 45, 2.75
Hypothesis 2: 45, 2.49
Hypothesis 3: 45, 2.52
Hypothesis 4: 45, 2.57
Hypothesis 5: 1000, 1.59
benchmark/openstacks/hindsight/30/P03
Correct hypothesis: 5
Hypothesis 0: 45, 1.74
Hypothesis 1: 45, 1.69
Hypothesis 2: 1000, 1.83
Hypothesis 3: 1000, 1.82
Hypothesis 4: 1000, 1.96
Hypothesis 5: 45, 1.71
benchmark/openstacks/hindsight/30/P04
Correct hypothesis: 4
Hypothesis 0: 1000, 1.12
Hypothesis 1: 1000, 1.37
Hypothesis 2: 1000, 1.16
Hypothesis 3: 1000, 1.10
Hypothesis 4: 46, 1.65
Hypothesis 5: 1000, 1.11